곱창이 먹고싶어서 만들었습니다.

smartstore.naver.com 주소를 가지는 네이버 스마트 스토어에서 사용 가능합니다.

남용하진 말아주세요.

## Notice

* 본인의 크롬 버전에 맞는 드라이버를 다운로드 하고 driver_path 값을 크롬드라이버 위치에 맞게 설정 해주세요.

* requirement를 모두 다운 받아 주세요.

* 다음 변수 값을 확인, 조정 하신 후 사용해 주세요.
  * `driver_path`
  * `d_time`
  * `order`
  * if ordering through excel: `input_path`
  * if using pay_with_point: `payment_pw`

* 반드시 로그인 부분에서 수동으로 로그인을 해 주고 넘어가셔야 합니다.

* pay_later()사용을 권장합니다. 주문완료 해놓고 나중에 수동으로 결제 할 수 있습니다.
  * 그걸 몰라서 pay_with_point를 만들어 버렸습니다.;;

* pay_with_point() 를 사용하기 위해선 학습된 tensorflow 모델이 필요합니다.
  * 제가 학습시켜놓은 모델 (h5파일)을 받아서 불러오기 해도 되고
  * 간단한 딥러닝을 배우고 싶으시면 gopket_Keypad_Analyzer를 참고하여 모델을 생성하고 학습시켜 주세요.

* 사용중 에러가 나면 해당 구간에서의 time.sleep() 을 조금 더 늘려 보세요.

## requirements
* python 3.7~
* selenium

if ordering through excel
* openpyxl

if using image classification for pay_with_point()
* tensorflow 2.3
* opencv-python (cv2)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime

## comment out if you won't import input values from gopketter_input.xlsx ##
import openpyxl

## comment out if you won't use pay_with_point() ##
import tensorflow as tf
import numpy as np
import cv2

def set_driver(headless = True, driver_path="C:/tutorial/chromedriver_86.exe"):
    options = Options()
    options.headless = headless # 별도 창 띄우지 않고 driver 실행 (PhantomJS 쓰지 않아도 됨)
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    return driver

In [2]:
### headless 사용시 봇 탐지 우회 ###
##ref. https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/

# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")

# # user agent
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
# user_agent = driver.find_element_by_css_selector('#user-agent').text
# print('User-Agent: ', user_agent)

# # language
# options.add_argument("lang=ko_KR")

In [3]:
### input 읽어오기 ###
input_path = r"C:\Users\lunat\Desktop\gopketter\gopketter_input.xlsx" #"../input/gopketter_input.xlsx"

## default input ##
order = [2,0,0,0,1,4] #소팔소곱창, 소팔한우곱창, 소막창, 소대창, 소대창, 소팔디핑소스
payment_pw, d_time =  "654321", datetime.datetime(2020,10,27,16,0,0)

if os.path.exists(input_path): 
    wb = openpyxl.load_workbook(input_path) #Read xl file
    print(f"{input_path}로 부터 인풋 값을 불러옵니다.")
    try:
        input_sheet = wb[f'Sheet1']
        
        order = [] #소팔소곱창, 소팔한우곱창, 소막창, 소대창, 소대창, 소팔디핑소스
        for row in input_sheet["C3":"C8"]:
            for cell in row:
                order.append(abs(cell.value))
        print(f"order: ", order)
        
        payment_pw = input_sheet["B11"].value
        if isinstance(input_sheet["B13"].value, (datetime.datetime, datetime.time)):
            d_time = input_sheet["B13"].value
        else:
            print("시간값을 정상적으로 읽어오지 못했습니다. 디폴트 값으로 유지됩니다.")
        print(f"d_time: {d_time}")
        
    except KeyError:
        print(f"{input_path} 에 'Sheet1' 시트가 존재하지 않습니다. 디폴트 값으로 주문합니다.")
else:
    print("%s 에 파일이 존재하지 않습니다." %input_path)

menu_name = "소팔소곱창 소팔한우곱창 소막창 소대창 소대창 소팔디핑소스".split(" ")
for count,name in zip(order, menu_name):
    print(f"{name:ㅡ<8} : {count: >2} 개")

# store_url, n_th = "https://smartstore.naver.com/haeyang4034/products/3368978129", 2 #테스트용 la갈비
if order[0]: store_url, n_th = "https://smartstore.naver.com/soffal/products/343944130", 1 #소곱창
else: store_url, n_th = "https://smartstore.naver.com/soffal/products/4938214965", 1 #한우곱창

### input 확인 ###
assert_mesasge = "소곱창/한우곱창 둘 중 하나는 수량이 0 이어야 합니다. 최대 수량은 4개 입니다."
assert order[0]*order[1]==0 and ((order[0] in [1,2,3,4]) or (order[1] in [1,2,3,4])), assert_message

C:\Users\lunat\Desktop\gopketter\gopketter_input.xlsx로 부터 인풋 값을 불러옵니다.
order:  [4, 0, 0, 1, 1, 8]
d_time: 16:00:00
소팔소곱창ㅡㅡㅡ :  4 개
소팔한우곱창ㅡㅡ :  0 개
소막창ㅡㅡㅡㅡㅡ :  0 개
소대창ㅡㅡㅡㅡㅡ :  1 개
소대창ㅡㅡㅡㅡㅡ :  1 개
소팔디핑소스ㅡㅡ :  8 개


In [4]:
driver = set_driver(headless=False)
login_url = "https://nid.naver.com"
driver.get(login_url)

print("로그인 해주세요")

로그인 해주세요


In [5]:
### 필요한 함수 미리 정의하기 ###
def apply_option(driver, option, price_at_nth_div=7, n_th=1):
    xpath = f"""//*[@id="content"]/div/div[2]/div[2]/fieldset/div[{price_at_nth_div-2}]/div[{n_th}]"""
    option_cnt = 0

    for index, amount in enumerate(option): #e.g. 1,1,0,4
        #print(f"index, amount : {index}, {amount}")

        #수량 확인-> 0개 처리.
        if amount == 0: continue

        #옵션종류 보는 버튼 클릭.
        option_view_btn = driver.find_element_by_xpath(xpath)
        option_view_btn.click()
        
        #옵션버튼 확인
        option_i_btn_xpath = f"""//*[@id="content"]/div/div[2]/div[2]/fieldset/div[{price_at_nth_div-2}]/div[{n_th}]/ul/li[{index+1}]"""
        #print(option_i_btn_xpath, option_cnt)
        option_i_btn = driver.find_element_by_xpath(option_i_btn_xpath)
        
        #품절 처리
        if option_i_btn.text[-3:-1] == "품절" :
            print("품절된 옵션입니다. 해당 옵션은 주문하지 않습니다.")
            option_view_btn.click() #
            continue

        #옵션버튼 클릭
        option_i_btn.click()
        option_cnt += 1

        #옵션 수량 + 버튼 클릭
        option_i_plus_btn_xpath = f"""//*[@id="content"]/div/div[2]/div[2]/fieldset/div[{price_at_nth_div-1}]/ul/li[{option_cnt+1}]/div/div/div/button[2]"""
        option_i_plus_btn = driver.find_element_by_xpath(option_i_plus_btn_xpath)
        for _ in range(amount-1):
            option_i_plus_btn.click()

def CountDown(d_time):
    """Waits until appointed time.
    
    0.2 seconds before the time, this function will return nothing so that the next code can run.
    
    takes datetime.date or datetime.time instance as argument:d_time
    """
    if isinstance(d_time, datetime.time): 
        today_d_time = (datetime.datetime.now().strftime("%Y %m %d").split(" ") + 
                        d_time.strftime("%H %M %S").split(" "))
        d_time = datetime.datetime(*tuple(int(i) for i in today_d_time))
    elif isinstance(d_time, datetime.datetime):
        pass
    else: raise TypeError
    print(f"WAIT UNTIL: {d_time}")
    
    while True:
        now = datetime.datetime.now()
            
        seconds_left = (d_time - now).total_seconds()
        if seconds_left < 0.2: return
        if seconds_left >= 10:
            print(f"Waiting...{int(seconds_left)} seconds left.\r", end="")
            time.sleep(1)
        else: 
            print(f"Waiting...{seconds_left:0.1f} seconds left.\r", end="")
            time.sleep(0.1)
            
def pay_later(driver):
    normal_pay_xpath = """//*[@id="orderForm"]/div/div[5]/div[1]/div[1]/ul/li[4]/div[1]/span/span"""
    
    driver.find_element_by_xpath(normal_pay_xpath).click() #일반결제
    time.sleep(0.2)
    #driver.implicitly_wait(0.5)
    driver.find_element_by_xpath("""//*[@id="orderForm"]/div/div[5]/div[1]/div[1]/ul/li[4]/ul/li[4]""").click() #나중에결제
    driver.find_element_by_xpath("""//*[@id="allAgree"]""").click() #전체동의 
    driver.find_element_by_xpath("""//*[@id="orderForm"]/div/div[7]/button""").click() #결제하기
    print("주문을 완료하였습니다. 결제내역에서 결제를 마저 진행 해 주세요.")

def pay_with_point(driver, model, password):
    driver.find_element_by_xpath("""//*[@id="allAgree"]""").click() #전체동의 
    driver.find_element_by_xpath("""//*[@id="orderForm"]/div/div[7]/button""").click() #결제하기
    
    #비밀번호 입력 창으로 전환
    time.sleep(1) #driver.implicitly_wait(1)
    old_window = driver.window_handles[0]
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)
    driver.save_screenshot("number_key_board.jpg") #스크린샷 저장
    
    #이미지 전처리
    screenshot_array = cv2.imread("number_key_board.jpg")
    col_ratio_points = [i/364 for i in (49,140,230,323)]
    row_ratio_points = [i/585 for i in (366,430,493)]

    row = [int(len(screenshot_array)*i) for i in row_ratio_points]  #0.579, 0.684, 0.789, 0.894
    col = [int(len(screenshot_array[0])*i) for i in col_ratio_points] # (0, 0.25, 0.5, 0.75, 1)
    crop_pxl = 16
    keypad_img_dict = dict.fromkeys([(i,j) for i in range(1,4) for j in range(1,5)], None)
    for i in range(1,4):
        for j in range(1,5):
            keypad_img_dict[(i,j)] = screenshot_array[row[i-1]-crop_pxl:row[i-1]+crop_pxl,
                                                      col[j-1]-crop_pxl:col[j-1]+crop_pxl]/255.0
    #숫자 예측
    key_dict = {}
    for key,value in keypad_img_dict.items():
        img = value.copy()
        pred = loaded_model.predict(img[np.newaxis,:,:,:])[0]
        key_dict[str(pred.argmax())] = key
    
    print(key_dict)
    
    #숫자 입력
    for i in password:
        r,c = key_dict[i]
        driver.find_element_by_xpath(f"""//*[@id="fm"]/div/div[2]/table/tbody/tr[{r}]/td[{c}]/button""").click()
        time.sleep(0.02)
    
    driver.switch_to.window(old_window)
    print("네이버페이 포인트로 주문을 완료하였습니다.")

In [ ]:
driver.get(store_url)

### 페이지 분석 ###
#가끔 카드 이벤트 등으로 div가 하나 더 생겨서 숫자가 밀림. 이를 방지하기 위해 div의 갯수를 세어주는게 좋음.
#이벤트 총금액의 div[n] 은 7임

price_at_nth_div = 7
while price_at_nth_div < 10:
    try:
        total_price_text_xpath = f"""//*[@id="content"]/div/div[2]/div[2]/fieldset/div[{price_at_nth_div}]/strong"""
        if driver.find_element_by_xpath(total_price_text_xpath).text[-2:] == "금액":
            break
        else: price_at_nth_div += 1
    except:
        price_at_nth_div += 1
print("price_at_nth_div", price_at_nth_div)

### Countdown 실행 ###
CountDown(d_time)


### 주문 실행 ###
cnt=1 #리프레시 카운트용
while True:
    driver.get(store_url) #새로고침
    time.sleep(0.2)
    
    # 구매버튼 있으면 구매하기
    try:
        buy_btn = driver.find_element_by_xpath(f"""//*[@id="content"]/div/div[2]/div[2]/fieldset/div[{price_at_nth_div+1}]/ul[1]/li[1]""")
        print(buy_btn.text + "버튼 확인")
        if buy_btn.text == "구매하기":
            #수량 + 버튼 클릭
            print("수량+ 버튼 확인")
            product_amount_plus_btn_xpath = f"""//*[@id="content"]/div/div[2]/div[2]/fieldset/div[{price_at_nth_div-1}]/ul/li[1]/div/div/div/button[2]"""
            product_amount_plus_btn = driver.find_element_by_xpath(product_amount_plus_btn_xpath)
            order_amount = order[0]+order[1] #어차피 둘 중 하나는 0이니까
            for _ in range(order_amount-1): #초기값이 1이므로 1빼야함.
                product_amount_plus_btn.click()
            
            #옵션 있으면 옵션 담고 클릭
            option = order[2:]
            print("option: ", option)
            if sum(option) != 0:
                apply_option(driver=driver, option=option, price_at_nth_div=price_at_nth_div, n_th=n_th) #소팔소에선 n_th=1로!! 
            
            #구매버튼 클릭 & 루프 빠져나오기
            buy_btn.click()
            break
    
    # 구매버튼 없으면 루프 재 진입->새로고침
    except Exception as e:
        error_message = "".join(e.__str__().split("\n"))
        print(error_message + f" 에러 발생. 현재 창에서 구매가 불가능 합니다. 시도 횟수: {cnt} \r", end="")
        cnt += 1
        continue

time.sleep(0.3)

pay_later(driver=driver)
# print("loading model")
# loaded_model = tf.keras.models.load_model("C:\\Users\\lunat\\Desktop\\weights\\number_classifier_v0.3_2020.10.22.01.54.h5")
# print("finished loading")
# pay_with_point(driver, loaded_model, payment_pw)

### 시간 되면 아래 코드 실행 ###
#새로고침 & 주문시작

price_at_nth_div 7
WAIT UNTIL: 2020-10-30 16:00:00


In [ ]:
old_window = driver.window_handles[0]
#new_window = driver.window_handles[1]
driver.switch_to.window(old_window)